<a href="https://colab.research.google.com/github/BarryLiu-97/Pytorch-Tutorial/blob/master/07_Softmax_Classifier_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
import torch
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim
import time

In [18]:
batch_size = 64
transform = transforms.Compose([
  transforms.ToTensor(),              #将数据转换为channel×width×height格式，为了更高效地进行运算
  transforms.Normalize((0.1307, ), (0.3081, ))  #均值和标准差，用于数据标准化，这是对MNIST进行计算后得到的结果，已经算好了
])

train_dataset = datasets.MNIST(root='../dataset/mnist',
                train=True, download=True,
                transform = transform)
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)

test_dataset = datasets.MNIST(root='../dataset/mnist',
                train=False, download=True,
                transform = transform)
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)

In [5]:
class Net(torch.nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.l1 = torch.nn.Linear(784, 512)
    self.l2 = torch.nn.Linear(512, 256)
    self.l3 = torch.nn.Linear(256, 128)
    self.l4 = torch.nn.Linear(128, 64)
    self.l5 = torch.nn.Linear(64, 10)

  def forward(self, x):
    x = x.view(-1, 784)   #数据本身是(N×1×28×28)地Tensor，
    x = F.elu(self.l1(x))
    x = F.elu(self.l2(x))
    x = F.elu(self.l3(x))
    x = F.elu(self.l4(x))
    return self.l5(x)  #最后一层不做激活

model = Net()
model.cuda()

Net(
  (l1): Linear(in_features=784, out_features=512, bias=True)
  (l2): Linear(in_features=512, out_features=256, bias=True)
  (l3): Linear(in_features=256, out_features=128, bias=True)
  (l4): Linear(in_features=128, out_features=64, bias=True)
  (l5): Linear(in_features=64, out_features=10, bias=True)
)

In [11]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)  #冲量值设置为0.5，优化训练过程

In [29]:
def train(epoch):
  running_loss = 0.0
  for batch_idx, data in enumerate(train_loader, 0):
    inputs, target = data
    inputs = inputs.cuda()
    target = target.cuda()
    optimizer.zero_grad()

    #forward + backward + update
    outputs = model(inputs)
    loss = criterion(outputs, target) 
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    if batch_idx % 300 ==299:  
      #300次迭代输出一次loss
      print('[%d, %5d] loss:%.3f' % (epoch + 1, batch_idx + 1, running_loss / 300)) 
      running_loss= 0.0

In [38]:
def test():
  correct = 0
  total = 0
  with torch.no_grad():#该缩进块中的代码不会计算梯度
    for data in test_loader:
      images, labels = data
      images = images.cuda() 
      labels = labels.cuda()
      outputs = model(images)
      _, predicted = torch.max(outputs.data, dim=1)
      predicted = predicted.cuda() #输出为列数为10，代表十个类别，取出每一行数值最大地那一列的下标，即获得该样本的类别，带下为(N×1)
      total += labels.size(0)   #取出的是N
      correct += (predicted == labels).sum().item()   #张量之间的比较运算
  print('Acc on test set:%d %%' % (100* correct / total))

In [39]:
if __name__ == '__main__':
  start = time.time()
  for epoch in range(10):
    train(epoch)
    test()
  end = time.time()
  print(str(end-start) + 's')

[1,   300] loss:0.179
[1,   600] loss:0.171
[1,   900] loss:0.155
Acc on test set:95 %
[2,   300] loss:0.146
[2,   600] loss:0.136
[2,   900] loss:0.128
Acc on test set:95 %
[3,   300] loss:0.124
[3,   600] loss:0.112
[3,   900] loss:0.112
Acc on test set:96 %
[4,   300] loss:0.103
[4,   600] loss:0.102
[4,   900] loss:0.096
Acc on test set:96 %
[5,   300] loss:0.091
[5,   600] loss:0.087
[5,   900] loss:0.083
Acc on test set:96 %
[6,   300] loss:0.077
[6,   600] loss:0.077
[6,   900] loss:0.077
Acc on test set:97 %
[7,   300] loss:0.064
[7,   600] loss:0.071
[7,   900] loss:0.068
Acc on test set:97 %
[8,   300] loss:0.057
[8,   600] loss:0.058
[8,   900] loss:0.063
Acc on test set:97 %
[9,   300] loss:0.050
[9,   600] loss:0.053
[9,   900] loss:0.056
Acc on test set:97 %
[10,   300] loss:0.046
[10,   600] loss:0.047
[10,   900] loss:0.048
Acc on test set:97 %
162.53466486930847s
